In [1]:
import os
import time
import pandas as pd
import pymysql
import numpy as np
from haversine import haversine
from decimal import Decimal
from tqdm.auto import tqdm

def dev_db_conn():
    """개발 DB 접속 함수"""
    conn = pymysql.connect(
        host = 'db-7ma06.pub-cdb.ntruss.com',
        user = 'redtable',
        password = 'fpemxpdlqmf5491!@#',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    return conn


def rt_db_conn():
    """빅데이터센터 DB 접속 함수"""
    conn = pymysql.connect(
        host = '220.95.212.6',
        user = 'red_dev_jiang',
        password = 'redtable211b',
        db = 'redtable_bigdata_center',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    conn.query("set character_set_connection=utf8;")
    conn.query("set character_set_server=utf8;")
    conn.query("set character_set_client=utf8;")
    conn.query("set character_set_results=utf8;")
    conn.query("set character_set_database=utf8;")
    return conn


file_path = os.path.join(os.environ["HOMEPATH"], "Downloads/")

# 식당 주변 숙소 데이터

# RT_HOTEL_NEARBY_RSTRNT_INFO_202407

## https://www.bigdata-culture.kr/bigdata/user/data_market/detail.do?id=ee2a1d50-3b97-11eb-af9a-4b03f0a582d6

In [2]:
with rt_db_conn() as conn:
    sql = """
    SELECT TABLE_NAME, COLUMN_NAME, column_comment
    FROM information_schema.`COLUMNS`
    WHERE TABLE_NAME = 'RT_HOTEL_NEARBY_RSTRNT_INFO_202407'
    """
    columns = pd.read_sql(sql, conn)
    
# 컬럼명 가져오기
db_cols = ','.join(columns['COLUMN_NAME'])

In [5]:
def text_extract(input_text):
    """상세주소 제외한 함수"""
    result_text = " ".join(input_text.split()[:-1])
    if result_text.endswith("리"):
        return " ".join(result_text.split()[:-1])
    return result_text

with dev_db_conn() as conn:
    sql = """
    SELECT T1.id, T1.name, 
        IF(T1.road_addr = '', NULL, T1.road_addr) AS 'road_addr',
        IF(T1.common_addr = '', NULL, T1.common_addr) AS 'common_addr',
        T1.latitude, T1.longitude, T1.category, 
        IF(T1.phone = '', NULL, REPLACE(T1.phone, '-', '')) AS 'phone'
    FROM redtable_raw_2022.naver_other T1
    WHERE T1.category IN ('호텔', '모텔', '게스트하우스', '펜션');
    """
    df_hotel = pd.read_sql(sql, conn)
    
df_hotel["검색주소"] = df_hotel["common_addr"].apply(lambda x:text_extract(x))

# 임의의 id 생성 (RSTR 테이블의 마지막 ID값으로 더해서 생성)
df_hotel["hotel_id"] = df_hotel.index + 300680045

In [6]:
df_hotel

,id,name,road_addr,common_addr,latitude,longitude,category,phone,검색주소,hotel_id
0,10971511,아침고요펜션,경기 가평군 상면 수목원로 380,경기도 가평군 상면 행현리 663-4,37.748418,127.354023,펜션,01089491927,경기도 가평군 상면,300680045
1,10982951,패션호텔,부산 부산진구 가야대로468번길 15,부산광역시 부산진구 개금동 177-29,35.152391,129.023460,모텔,0518921501,부산광역시 부산진구 개금동,300680046
2,10984523,약수천사우나모텔,인천 강화군 길상면 강화동로 23,인천광역시 강화군 길상면 온수리 500-22,37.641446,126.491247,모텔,0329377511,인천광역시 강화군 길상면,300680047
3,10988738,백제모텔,전남 무안군 무안읍 무안로 469 백제모텔,전라남도 무안군 무안읍 성남리 187-1,34.986272,126.476337,모텔,0614538080,전라남도 무안군 무안읍,300680048
4,11068795,자우림펜션,경기 가평군 북면 가화로 2697-137,경기도 가평군 북면 적목리 153,37.970315,127.451632,펜션,None,경기도 가평군 북면,300680049
...,...,...,...,...,...,...,...,...,...,...
1110,1992272104,미담하우스,제주 제주시 도두봉6길 9-3,제주특별자치도 제주시 도두일동 1700-9,33.508300,126.472540,펜션,None,제주특별자치도 제주시 도두일동,300681155
1111,1992958240,제이펜션,경기 가평군 상면 청군로 607,경기도 가평군 상면 임초리 42-1,37.771492,127.380583,펜션,None,경기도 가평군 상면,300681156
1112,1995640109,고기리 다온스테이,경기 용인시 수지구 호수로52번길 51 1동 1층 1호,경기도 용인시 수지구 고기동 30-68 1동 1층 1호,37.352295,127.067408,펜션,None,경기도 용인시 수지구 고기동 30-68 1동 1층,300681157
1113,1996439912,바닷가로뎀나무민박,전북 부안군 진서면 작당길 19,전북특별자치도 부안군 진서면 운호리 276,35.586919,126.547023,펜션,01086339046,전북특별자치도 부안군 진서면,300681158


In [7]:
for i in tqdm(range(len(df_hotel))):
    try:
        with dev_db_conn() as conn:
            with conn.cursor() as curs:
                sql = f"""
                SELECT %s AS 'HOTEL_ID', %s AS 'HOTEL_NM', %s AS 'HOTEL_LNM_ADDR', %s AS 'HOTEL_ROAD_NM_ADDR',
                   %s AS 'HOTEL_LA', %s AS 'HOTEL_LO', %s AS 'HOTEL_TEL_NO',
                   T3.SRC_ID AS 'RSTRNT_ID', T1.RSTR_NM AS 'RSTRNT_NM',
                   IF(T1.RSTR_LNNO_ADRES = '', NULL, T1.RSTR_LNNO_ADRES) AS 'RSTRNT_LNM_ADDR',
                   IF(T1.RSTR_RDNMADR = '', NULL, T1.RSTR_RDNMADR) AS 'RSTRNT_ROAD_NM_ADDR',
                   T1.RSTR_LA AS 'RSTRNT_LA', T1.RSTR_LO AS 'RSTRNT_LO',
                   'KOR' AS '국가명', 
                   REPLACE(REPLACE(REPLACE(T1.AREA_NM_1, '강원도', '강원특별자치도'), '전라북도', '전북특별자치도'), '제주도', '제주특별자치도') AS 'CTY_NM',
                   '' AS 'CFR_HSDL_STATN_NM', '' AS 'HOTEL_RSTRNT_BTWN_DSTNC_CO'
                FROM redtable01.RSTR T1
                INNER JOIN redtable01.RSTR_INDEX T2 ON T1.ID = T2.RSTR_ID
                INNER JOIN redtable01.RSTR_CONN T3 ON T2.RSTR_ID = T3.RSTR_ID
                WHERE T2.IS_USE=1 AND T2.IS_OPEN=1 AND T3.SRC='live_db' AND T1.AREA_NM_1 IS NOT NULL AND T1.AREA_NM_1<>''
                  AND T1.RSTR_NM IS NOT NULL AND T1.RSTR_NM<>'' AND T1.RSTR_NM NOT LIKE '%%？%%'
                  AND T1.RSTR_LNNO_ADRES LIKE %s
                GROUP BY T3.SRC_ID
                """
                curs.execute(sql, (
                    df_hotel["hotel_id"][i], df_hotel["name"][i], df_hotel["common_addr"][i], df_hotel["road_addr"][i],
                    df_hotel["latitude"][i], df_hotel["longitude"][i], df_hotel["phone"][i], f'%{df_hotel["검색주소"][i]}%'))
                df_result = pd.DataFrame(curs.fetchall())

        if i == 0:
            df_final = df_result

        else:
            df_final = pd.concat([df_final, df_result])

        time.sleep(2)
    except:
        time.sleep(10)
        
# 결측치 및 불필요한 데이터 정리
df_final = df_final.fillna(np.nan)
df_final.replace({np.nan:None, '\\n':None}, inplace=True)

  0%|          | 0/1115 [00:00<?, ?it/s]

In [8]:
def haversine_calculate(store_la,store_lo,airport_la,airport_lo):
    """위경도로 거리 검색하는 함수"""
    try:
        store = (float(store_la), float(store_lo))
        ariport = (float(airport_la), float(airport_lo))
        return haversine(store, ariport, unit = 'm')
    except:
        return 0

# 함수를 통해 위경도로 거리 검색하기 (단위 m)
df_final["HOTEL_RSTRNT_BTWN_DSTNC_CO"] = df_final.apply(lambda row:
                                                      (haversine_calculate(row["HOTEL_LA"],row["HOTEL_LO"]
                                                                           ,row["RSTRNT_LA"],row["RSTRNT_LO"]) ),axis=1).tolist()

# 주소가 이상한 결과 필터링
df_final = df_final[df_final["HOTEL_RSTRNT_BTWN_DSTNC_CO"] != 0]

# 거리가 1000m 이하 매장만 추출
df_final = df_final[df_final["HOTEL_RSTRNT_BTWN_DSTNC_CO"] < 1000]
df_final.reset_index(drop=True, inplace=True)

In [9]:
# 근처 고속철도 넣기
train = pd.read_excel(f"{file_path}고속철도역명_20240712.xlsx", sheet_name="고속철도명")

train = train[["고속철도명", "고속철도 주소검색"]]
train.drop_duplicates(subset=["고속철도명"], inplace=True)
train.reset_index(drop=True, inplace=True)

def find_high_speed_rail_name(address):
    for i in range(len(train)):
        if train["고속철도 주소검색"][i] in address:
            return train["고속철도명"][i]
    return None

df_final["CFR_HSDL_STATN_NM"] = df_final["RSTRNT_LNM_ADDR"].apply(find_high_speed_rail_name)

FileNotFoundError: [Errno 2] No such file or directory: '\\Users\\opjet\\Downloads/고속철도역명_20240712.xlsx'

In [ ]:
with rt_db_conn() as conn:
    with conn.cursor() as curs:
        sql = f"""
        INSERT INTO redtable_bigdata_center.RT_HOTEL_NEARBY_RSTRNT_INFO_202407({db_cols})
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """
        val = df_final.apply(lambda row:(row["HOTEL_ID"], row["HOTEL_NM"], row["HOTEL_LNM_ADDR"], row["HOTEL_ROAD_NM_ADDR"],
                                  row["HOTEL_LA"], row["HOTEL_LO"], row["HOTEL_TEL_NO"], row["RSTRNT_ID"],
                                  row["RSTRNT_NM"], row["RSTRNT_LNM_ADDR"], row["RSTRNT_ROAD_NM_ADDR"],row["RSTRNT_LA"],
                                  row["RSTRNT_LO"], row["국가명"], row["CTY_NM"],row["CFR_HSDL_STATN_NM"],row["HOTEL_RSTRNT_BTWN_DSTNC_CO"]), 
                                  axis=1).tolist()
        curs.executemany(sql, val)